In [14]:
import os
import math
import json
from parser import proc_xml
from roman_to_symbol import proc_roman_to_symbol
from to_pianoroll import proc_event_to_midi, proc_midi_to_pianoroll

In [24]:
root_xml = 'dataset/xml'
root_event = 'dataset/event'
root_pianoroll = 'dataset/pianoroll'
log_dir = 'log'


def traverse_dir(root_dir, extension='.xml', is_pure=True):
    print('[*] Scanning...')
    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(extension):
                mix_path = os.path.join(root, file)
                pure_path = mix_path[len(root_dir)+1:] if is_pure else mix_path
                file_list.append(pure_path)

    return file_list


def error_handler(e, url, log, side_info=''):
    print(e)
    print(url, '\n')
    log.write(side_info + '  -  '+str(e) + '\n' + url + '\n')


def split_file(path):
    base = os.path.basename(path)
    filename = base.split('.')[0]
    extension = base.split('.')[1]
    return path[:-len(base)-1], filename, extension


def proc(xml_list, index=0):
    num_xml = len(xml_list)
    log = open(os.path.join(log_dir, 'log_'+str(index)+'.txt'), 'w')
        
    cnt_ok = 1
    cnt_total = 0
    for xidx in range(num_xml):  # num_xml
        cnt_total += 1
        path, fn, _ = split_file(xml_list[xidx])
        tmp = path.split('/')
        url = 'https://www.hooktheory.com/theorytab/view/' + tmp[-2] + '/' + tmp[-1]
        try:    
            current = '[%d] (%d/%d)[%d/%d] %s' % (index, xidx, num_xml, cnt_ok, cnt_total, xml_list[xidx])
            print(current)

            # path arrangement
            path_xml = os.path.join(root_xml, path)
            path_event = os.path.join(root_event, path)
            path_pianoroll = os.path.join(root_pianoroll, path)

            # roman
            xml_file = os.path.join(path_xml, fn+'.xml')
            name = fn + '_roman'

            try:
                raw_roman = proc_xml(xml_file, save_path=path_event, name=name, save_type='json')
            except Exception as e:
                print('> Broken File!!!')
                error_handler(e, url, log, current)
                continue

            # to event symbol
            name = fn + '_symbol_key'
            raw_symbol_key = proc_roman_to_symbol(raw_roman, save_path=path_event, name=name, save_type='json', is_key=True)
            name = fn + '_symbol_nokey'
            raw_symbol_nokey = proc_roman_to_symbol(raw_roman, save_path=path_event, name=name, save_type='json', is_key=False)

            # to midi
            _ = proc_event_to_midi(raw_symbol_key, save_path=path_pianoroll, name=fn+'_key')
            midi = proc_event_to_midi(raw_symbol_nokey, save_path=path_pianoroll, name=fn+'_nokey')

            # to pianoroll

            beats_in_measure = int(raw_symbol_nokey['metadata']['beats_in_measure'])
            pianoroll = proc_midi_to_pianoroll(midi, beats_in_measure)
            pianoroll.save(os.path.join(path_pianoroll, fn+'.npz'))

            # plot
            pianoroll.plot()
            plt.savefig(os.path.join(path_pianoroll, fn+'.png'), dpi=500)
            plt.close()

            cnt_ok += 1
        except Exception as e:
            error_handler(e, url, log)

    print('Done!!!')
    log.close()
    queue.put((cnt_ok, cnt_total))

In [25]:
# xml_list = traverse_dir(root_xml)

# with open('xml_list.json', "w") as f:
#     json.dump(xml_list, f)
        
with open('xml_list.json', "r") as f:
    xml_list = json.load(f)
    
proc(xml_list[0:2], 0)

[0] (0/2)[1/1] u/uesaka-sumire/nanatsu-no-umi-yori-kimi-no-umi/chorus.xml
name 'plt' is not defined
https://www.hooktheory.com/theorytab/view/uesaka-sumire/nanatsu-no-umi-yori-kimi-no-umi 

[0] (1/2)[1/2] u/uesaka-sumire/nanatsu-no-umi-yori-kimi-no-umi/verse-and-pre-chorus.xml
name 'plt' is not defined
https://www.hooktheory.com/theorytab/view/uesaka-sumire/nanatsu-no-umi-yori-kimi-no-umi 

Done!!!


NameError: name 'queue' is not defined